In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import leafmap.foliumap as leafmap
from IPython.display import display
import ipywidgets as widgets
import matplotlib.transforms as transforms

# Load the processed data
df = pd.read_csv('4s_5s.csv')
df['OBSERVATION DATE'] = pd.to_datetime(df['OBSERVATION DATE'])  # Ensure datetime format

# Drop duplicates to keep only the first observation of each species per year
first_observations = df.drop_duplicates(subset=['COMMON NAME', 'YEAR'], keep='first')
def interactive_bird_observations(df):
    df.reset_index(drop=True, inplace=True)  # Reset the DataFrame index
    species_list = sorted(df['COMMON NAME'].unique())
    dropdown = widgets.Dropdown(
        options=species_list,
        value=species_list[0],
        description='Species:',
        disabled=False,
    )

    output_map = widgets.Output()
    output_plot = widgets.Output()

    def update_visuals(species):
        # Update scatter plot
        output_plot.clear_output(wait=True)
        with output_plot:
            species_df = df[df['COMMON NAME'] == species].copy()
            species_df['OBSERVATION DATE'] = pd.to_datetime(species_df['OBSERVATION DATE'])
            species_df['DAY OF YEAR'] = species_df['OBSERVATION DATE'].dt.dayofyear
            fig, ax = plt.subplots(figsize=(8, 0.3))
            ax.scatter(species_df['DAY OF YEAR'], np.zeros_like(species_df['DAY OF YEAR']), alpha=0.5, color='blue')
            setup_plot(ax, species)
            plt.show()

        # Update map
        output_map.clear_output(wait=True)
        with output_map:
            m = leafmap.Map(center=[df['LATITUDE'].mean(), df['LONGITUDE'].mean()], zoom=10)
            filtered_data = df[df['COMMON NAME'] == species].copy()
            if not filtered_data.empty:
                m.add_points_from_xy(
                    data=filtered_data, 
                    x="LONGITUDE", 
                    y="LATITUDE", 
                    color="red", 
                    fill_color="blue",
                    size=5
                )
            display(m)

    def setup_plot(ax, species):
        current_date = datetime.datetime.now()
        current_day_of_year = current_date.timetuple().tm_yday
        week_start = current_day_of_year - current_date.weekday()
        week_end = week_start + 7
    
        # Original tick positions for months
        month_ticks = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335, 366]
        # Month names for labels
        month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan']
    
        # Calculate midpoints to position the labels
        midpoints = [(month_ticks[i] + month_ticks[i + 1]) / 2 for i in range(len(month_ticks) - 1)]
    
        # Set the ticks without labels
        ax.set_xticks(month_ticks)
        ax.set_xticklabels([])  # Hide default numerical labels
    
        # Set custom tick labels manually positioned to the midpoints
        for i, label in enumerate(month_labels[:-1]):  # Exclude the last redundant 'Jan' label
            trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
            ax.text(midpoints[i], 0, label, transform=trans, ha='center', va='top', rotation=0)
    
        # Add red highlight for the current week
        ax.axvspan(week_start, week_end, color='red', alpha=0.2)
    
        # Plot settings
        ax.set_title(species, loc='left', fontsize=10, verticalalignment='top')
        ax.set_yticks([])
        ax.set_xlim(1, 366)
        ax.grid(True)


    dropdown.observe(lambda change: update_visuals(change.new), names='value')
    display(dropdown, output_plot, output_map)
    update_visuals(dropdown.value)  # Initial visualization

# Assuming 'first_observations' is your DataFrame
interactive_bird_observations(df)


Dropdown(description='Species:', options=("Allen's Hummingbird", 'American Golden-Plover', 'American Redstart'…

Output()

Output()

In [12]:
!pipreqsnb --force "/home/hydrospheric/PERSONAL/CODE/BIRDS/eBird_yolo/webapp/"

pipreqs  --force /home/hydrospheric/PERSONAL/CODE/BIRDS/eBird_yolo/webapp/
INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in /home/hydrospheric/PERSONAL/CODE/BIRDS/eBird_yolo/webapp/requirements.txt
